Download dependencies:

In [ ]:
!pip install zipp
!pip install requests
!pip install pandas

Import Dependencies:

In [18]:
from zipfile import ZipFile
import requests
import pandas as pd
import plotly as plot
import plotly.express as px
import numpy as np

Download the file:

In [3]:
url = 'https://download-data.deutschebahn.com/static/datasets/wifi/20171212_wifionice.zip'
response = requests.get(url)

with open('20171212_wifionice.zip', 'wb') as f:
    f.write(response.content)

Extract the csv file and read dataset:

In [4]:
with ZipFile('20171212_wifionice.zip', 'r') as zip_file:
    zip_file.extract('surveyor_hackathon_data_20171212.csv')

data = pd.read_csv('surveyor_hackathon_data_20171212.csv', sep=';', low_memory=False)

remove missing locations

In [5]:
data = data.dropna(subset=["gps_breite"])
data = data.dropna(subset=["gps_laenge"])

remove impossible speeds

In [6]:
data = data.drop(data[data["gps_v"] > 85].index)
data = data.dropna(subset=["gps_v"])

set transfer speed NaN to 0

In [7]:
data['tprx'] = data['tprx'].fillna(value=0)
data['tptx'] = data['tptx'].fillna(value=0)

pax_auth NaN

In [8]:
data = data.dropna(subset=["pax_auth"])

drop unnecessary columns

In [9]:
data = data.drop(['gps_hoehe', 'sat', 'gps_richtung', 'pax_total', 'link_id', 'link_gw_conn'], axis=1)

In [10]:
data.columns = ['sid', 'created', 'gps_width', 'gps_length', 'speed', 'devicecount', 'received_byte', 'sent_byte', 'ping']

In [11]:
data['created'] = pd.to_datetime(data['created'])
data = data.groupby(['sid', pd.Grouper(key='created', freq='1Min')]).first().reset_index()

In [ ]:
data['sid'] = data['sid'].astype(int)
data['devicecount'] = data['devicecount'].astype(int)
data

In [13]:
data.to_csv('clean_data.csv', index=False)

In [ ]:
data = pd.read_csv('clean_data.csv')
data

Changes by Niklas

In [ ]:
data['sent_byte_per_user'] = data['sent_byte']/data['devicecount']
fig_sent_byte = px.scatter(data, x='sent_byte_per_user', y='devicecount', title='Sent Bytes Scatter Plot')
fig_sent_byte.update_xaxes(type='log')
fig_sent_byte.update_traces(marker=dict(size=2))
fig_sent_byte.show()

In [ ]:
fig_speed = px.scatter(data, x='received_byte', y='speed', title='Received Bytes per speed')
fig_speed.update_xaxes(type='log')
fig_speed.update_traces(marker=dict(size=2))
fig_speed.show()

In [ ]:
data['down_byte_per_user'] = data['received_byte'] / data['devicecount']
fig_down_byte = px.scatter(data, x='down_byte_per_user', y='devicecount', title='Download Bytes Scatter Plot')
fig_down_byte.update_xaxes(type='log')
fig_down_byte.update_traces(marker=dict(size=2))
fig_down_byte.show()